In [1]:
%load_ext autoreload
%autoreload 2

import __confing
__confing.change_to_root_folder()

Current directory: c:\Users\Leo\OneDrive\Escritorio\Final Proyect


In [2]:
try:
    __confing.execute_notebook('notebooks/data_extract.ipynb')
except:
    pass

In [3]:
import pandas as pd
import numpy as np

from src import value_filler as vf
from src import feature_selection as fs
from src import data_procces as dp

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from src import evaluation

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error,roc_auc_score,confusion_matrix
from sklearn.model_selection import KFold

In [4]:
def feature_importance(model, X: pd.DataFrame, y: pd.Series, n: int):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    feature_names_original = list(X.columns)
    
    target_name = y.name  # Obtener el nombre de la columna objetivo desde la Serie y
    
    if target_name in feature_names_original:
        feature_names_original.remove(target_name)
    
    max_importance = list(indices[:n])
    
    nueva_lista = [feature_names_original[i] for i in max_importance]
    
    return nueva_lista


In [5]:
n='a'
df=pd.read_csv(r'data\custom\all_waves.csv')
target=f'p{n}hosp1y'

In [6]:
selected = ['pahosp1y','paurina2y','pacholst','pabreast','paprost','papapsm','pamammog','pacage','padrinkcr','padrinkbd','padrinkb','paheight','paweight','pabmi','papaina','papainlv','papainfr','pafatigue','pawheeze','pabreath_m','paswell','pahearaid','pahearing','pasight','paglasses','palunglmt_m','pahrtatlmt','pastroklmt','paarthlmt','palowermoba','pauppermoba','pagrossaa','palgmusaa','pamobilaa','paclims','pasit','paarms','palift','pastoop','pachair','pameals','pashop','pamoney','pameds','pabedhlp','pabed','patoilt','paeat','pabath','pawalkr','padress','pahipcomp','pafall','pafallinj','pamhip','pahipe_m','parxarthr','parxstrok','parxhrtat','pacncrothr','pacncrmeds','pacncrradn','pacncrsurg','pacncrchem','parxdiab','parxdiabo','parxhibp','parechrtatt','parifaany','parfaany','parfcaren','pararcare','padresshlp', 'pawalkhlp', 'pabathehlp', 'paeathlp', 'pabedhlp', 'patoilethlp', 'pamealhlp', 'pashophlp', 'pamedhlp', 'pamoneyhlp','pahibpe','pagender','padiabe','pacancre','palunglmt_m','parxlung_m','pahrtatte','pastroke','paarthre','parifaany','parafaany','pahigov','pasmokev','pasmoken','paoangry','paosleep','paodngr','paodngr','paopace','paoplot','paoalchl','pawthh','paagey','pamomage','padadage','paprmem','parjudg','parorgnz']

In [7]:
selected = list(set(selected))
df = df.dropna(subset=[target]) 
df = df[selected].copy()

In [8]:
porcentaje_nulos = df.isnull().sum(axis=1) / len(df.columns)
df = df[(df[target] != 0) | (porcentaje_nulos <= 0.4)]

In [9]:
# fill
y = df[target]
X = df.drop(target,axis=1)

In [10]:
from src import feature_selection as fs
X = fs.fast_fill(X)

c:\Users\Leo\OneDrive\Escritorio\Final Proyect\src\feature_selection.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[columns] = dataframe[columns].fillna(0, inplace=True)


In [11]:
# split
X_train, X_test, y_train, y_test=dp.split_data(X, y, test_size=0.2, random_state=40)

In [12]:
# # resample 
X_train, y_train = dp.apply_resample(X_train, y_train,v=2)

In [13]:
# smote
X_train, y_train = dp.apply_smote(X_train, y_train)

In [14]:
# scaler
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)

In [15]:
model = XGBClassifier()
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(f'AUC={auc}')
print(y_test.value_counts())

AUC=0.8198813601322612
pahosp1y
0.0    15548
1.0     2827
Name: count, dtype: int64


In [16]:
print(feature_importance(model, X, y, 65))

['pagrossaa', 'palift', 'pashop', 'parifaany', 'pacholst', 'pabreath_m', 'pafatigue', 'pararcare', 'paswell', 'padiabe', 'pafall', 'parfcaren', 'parxhibp', 'pahrtatte', 'paurina2y', 'padrinkcr', 'pafallinj', 'parxdiabo', 'parxdiab', 'pacncrchem', 'pabathehlp', 'pawalkr', 'paprost', 'pashophlp', 'pahipcomp', 'pacncrmeds', 'pahigov', 'paeat', 'pahipe_m', 'paodngr', 'parxlung_m', 'pabath', 'pacncrsurg', 'padrinkb', 'parxhrtat', 'pauppermoba', 'papapsm', 'pacancre', 'pagender', 'pastroke', 'pawheeze', 'parxarthr', 'pasmokev', 'pamammog', 'papainfr', 'pachair', 'pabed', 'parfaany', 'pamobilaa', 'parxstrok', 'papaina', 'palunglmt_m', 'parafaany', 'palgmusaa', 'paeathlp', 'pameds', 'pacncrradn', 'pasmoken', 'pacage', 'pahrtatlmt', 'paarms', 'parechrtatt', 'pabedhlp', 'palowermoba', 'pameals']


In [17]:
print(feature_importance(model, X, y, 70))

['pagrossaa', 'palift', 'pashop', 'parifaany', 'pacholst', 'pabreath_m', 'pafatigue', 'pararcare', 'paswell', 'padiabe', 'pafall', 'parfcaren', 'parxhibp', 'pahrtatte', 'paurina2y', 'padrinkcr', 'pafallinj', 'parxdiabo', 'parxdiab', 'pacncrchem', 'pabathehlp', 'pawalkr', 'paprost', 'pashophlp', 'pahipcomp', 'pacncrmeds', 'pahigov', 'paeat', 'pahipe_m', 'paodngr', 'parxlung_m', 'pabath', 'pacncrsurg', 'padrinkb', 'parxhrtat', 'pauppermoba', 'papapsm', 'pacancre', 'pagender', 'pastroke', 'pawheeze', 'parxarthr', 'pasmokev', 'pamammog', 'papainfr', 'pachair', 'pabed', 'parfaany', 'pamobilaa', 'parxstrok', 'papaina', 'palunglmt_m', 'parafaany', 'palgmusaa', 'paeathlp', 'pameds', 'pacncrradn', 'pasmoken', 'pacage', 'pahrtatlmt', 'paarms', 'parechrtatt', 'pabedhlp', 'palowermoba', 'pameals', 'pasit', 'paopace', 'pahearaid', 'pamoney', 'padresshlp']


In [18]:
print(feature_importance(model, X, y, 75))

['pagrossaa', 'palift', 'pashop', 'parifaany', 'pacholst', 'pabreath_m', 'pafatigue', 'pararcare', 'paswell', 'padiabe', 'pafall', 'parfcaren', 'parxhibp', 'pahrtatte', 'paurina2y', 'padrinkcr', 'pafallinj', 'parxdiabo', 'parxdiab', 'pacncrchem', 'pabathehlp', 'pawalkr', 'paprost', 'pashophlp', 'pahipcomp', 'pacncrmeds', 'pahigov', 'paeat', 'pahipe_m', 'paodngr', 'parxlung_m', 'pabath', 'pacncrsurg', 'padrinkb', 'parxhrtat', 'pauppermoba', 'papapsm', 'pacancre', 'pagender', 'pastroke', 'pawheeze', 'parxarthr', 'pasmokev', 'pamammog', 'papainfr', 'pachair', 'pabed', 'parfaany', 'pamobilaa', 'parxstrok', 'papaina', 'palunglmt_m', 'parafaany', 'palgmusaa', 'paeathlp', 'pameds', 'pacncrradn', 'pasmoken', 'pacage', 'pahrtatlmt', 'paarms', 'parechrtatt', 'pabedhlp', 'palowermoba', 'pameals', 'pasit', 'paopace', 'pahearaid', 'pamoney', 'padresshlp', 'padress', 'pabmi', 'papainlv', 'paagey', 'patoilt']


In [19]:
print(feature_importance(model, X, y, 80))

['pagrossaa', 'palift', 'pashop', 'parifaany', 'pacholst', 'pabreath_m', 'pafatigue', 'pararcare', 'paswell', 'padiabe', 'pafall', 'parfcaren', 'parxhibp', 'pahrtatte', 'paurina2y', 'padrinkcr', 'pafallinj', 'parxdiabo', 'parxdiab', 'pacncrchem', 'pabathehlp', 'pawalkr', 'paprost', 'pashophlp', 'pahipcomp', 'pacncrmeds', 'pahigov', 'paeat', 'pahipe_m', 'paodngr', 'parxlung_m', 'pabath', 'pacncrsurg', 'padrinkb', 'parxhrtat', 'pauppermoba', 'papapsm', 'pacancre', 'pagender', 'pastroke', 'pawheeze', 'parxarthr', 'pasmokev', 'pamammog', 'papainfr', 'pachair', 'pabed', 'parfaany', 'pamobilaa', 'parxstrok', 'papaina', 'palunglmt_m', 'parafaany', 'palgmusaa', 'paeathlp', 'pameds', 'pacncrradn', 'pasmoken', 'pacage', 'pahrtatlmt', 'paarms', 'parechrtatt', 'pabedhlp', 'palowermoba', 'pameals', 'pasit', 'paopace', 'pahearaid', 'pamoney', 'padresshlp', 'padress', 'pabmi', 'papainlv', 'paagey', 'patoilt', 'padrinkbd', 'pasight', 'pabreast', 'paarthre', 'pamhip']


In [ ]:
raise ValueError('error')

In [ ]:
def procesar_dataframe(df, columnas):
    df_suma = df[columnas].sum(axis=1)
    df = df.drop(columns=columnas)
    df['nueva_columna'] = np.where(df_suma != 0, 1, np.nan)
    return df